## Setup

Kita udah nyediain array NumPy di bawah ini. Atribut dtype ngasih tau kita tipe data dari array tersebut. Atribut itemsize ngasih tau ukuran tiap item di array dalam byte.

Jalanin cell di bawah buat ngecek hasil output-nya.

In [1]:
import numpy as np
numbers = np.array([1.23456, 2.3456, 3.456, 7.56, 15.6745], dtype = 'float32')
print("Data type: ", numbers.dtype)
print("Bytes needed per element: ", numbers.itemsize)

Data type:  float32
Bytes needed per element:  4


## Mengubah Tipe Data di NumPy

Kita udah nulis kode di bawah ini buat ngubah angka-angka jadi array tipe FP16 pake fungsi np.float16. Jalanin cell berikut buat lihat gimana perubahan angka-angka tersebut.

## Float 16

In [2]:
f16_numbers = np.float16(numbers)
f16_bytes_per_element = f16_numbers.itemsize
print(f16_numbers)
print("Bytes needed per element: ", f16_bytes_per_element)

[ 1.234  2.346  3.455  7.56  15.67 ]
Bytes needed per element:  2


## Integer 8

In [3]:
### YOUR SOLUTION HERE ###
int8_numbers = np.int8(numbers)
int8_bytes_per_element = int8_numbers.itemsize
print(int8_numbers)
print("Bytes needed per element: ", int8_bytes_per_element)

[ 1  2  3  7 15]
Bytes needed per element:  1


## Quantization in Numpy

Asumsi Skenario:

Sekarang kita bakal anggap array numbers dari checkpoint sebelumnya itu isinya weights dari model deep learning. Dan kita bakal nyobain fungsi simple buat quantize array itu.

Metode yang tadi sebenernya udah ngubah nilai di array jadi integer, tapi masih bisa dibikin lebih bagus. Soalnya cara itu cuma pembulatan ke bawah, jadi informasi dari array aslinya bisa ilang. Contohnya nih: kalau semua weight nilainya < 1, terus dikonversi ke np.int8, semuanya jadi 0 dong! 😭

Biar hasilnya lebih akurat dan gak ngawur, kita perlu memperhitungkan range dari nilai di array aslinya dibandingin sama range angka yang bisa diwakilin oleh tipe data tujuan.

Solusinya:

Kita udah nulis fungsi simple buat quantize array pake metode yang disebut Absmax Quantization. Fungsi ini udah siap dipake di cell berikut.

Jalanin aja cell-nya buat nge-define fungsinya dulu 💪

In [4]:
def quantize(x):
    # scaling the values
    scale = 127 / max(abs(x))
    # quantizing
    quantized_value = (scale * x).round()
    return quantized_value.astype(np.int8)

In [5]:
quantized_array = quantize(numbers)
print(quantized_array)

[ 10  19  28  61 127]


## Quantization di Hugging Face

Kita barusan berhasil quantize array dari float32 (yang butuh 4 byte per elemen) jadi int8 (yang cuma butuh 1 byte per elemen)! 🔥

Sekarang coba bayangin kalo kita ngelakuin hal yang sama ke model transformer gede, yang punya bobot (weight matrix) segede gaban. Seberapa banyak memori komputasi model itu bisa dipangkas? 🤯

Untungnya, kita nggak perlu overthinking karena Hugging Face udah nyediain solusi praktis lewat library bitsandbytes. Dokumentasinya bisa lo cek di sini:
[https://huggingface.co/docs/bitsandbytes/main/en/index](https://huggingface.co/docs/bitsandbytes/main/en/index)

Kita juga udah nyiapin kode buat ngitung ukuran memori model distilgpt2 — versi yang lebih ramping dari GPT-2. Langsung aja jalanin cell-nya buat liat seberapa besar ukuran modelnya dalam satuan byte.

In [6]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
torch.manual_seed(19)

model_id = 'distilgpt2'
model = AutoModelForCausalLM.from_pretrained(model_id)
print("Model size in bytes: ", model.get_memory_footprint())

c:\Users\Radit\anaconda3\envs\environment\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


ModuleNotFoundError: No module named 'torch'

Tambahin dua argumen lagi ke metode from_pretrained biar kita bisa nge-load versi 8-bit quantized dari model distilgpt2:

Set device_map ke 'auto'

Set load_in_8bit ke True

Terus jangan lupa buat uncomment baris yang nge-print ukuran model-nya, dan jalanin cell-nya.

In [ ]:
int8_quantized_model = AutoModelForCausalLM.from_pretrained(model_id,
                                                            ### YOUR SOLUTION HERE ###
                                                            #device_map=  ,
                                                            #
                                             )
#print("8-bit Quantized model size in bytes: ", int8_quantized_model.get_memory_footprint())